In [25]:
import pandas as pd
from pathlib import Path

In [26]:
# Defining Paths
population_path = Path("../Group3Project1/Resources/sub-est2022.csv")
snp500_path = Path("../Group3Project1/Resources/constituents.csv")

# Reading CSV
population_data = pd.read_csv(population_path, header=0) 
snp500_data = pd.read_csv(snp500_path)

In [27]:
# Preparing and Cleaning SUMLEV 162(Cities) Population Data
cities_population_data = population_data[population_data["SUMLEV"] == 162]
cities_population_data = cities_population_data[['NAME', 'STNAME', 'POPESTIMATE2022']]
cities_population_data.columns = ['city', 'state', 'population']
cities_population_data['city'] = cities_population_data['city'].str.replace(
  r' (City|city|town|village|borough)', '', regex=True)
cities_population_data['city'] = cities_population_data['city'].str.replace(r'\bSt\. \b', 'Saint ', regex=True)

In [28]:
# Preparing and Cleaning SUMLEV 050(County) Population Data
counties_population_data = population_data[population_data["SUMLEV"] == 50]
counties_population_data = counties_population_data[['NAME', 'STNAME', 'POPESTIMATE2022']]
counties_population_data.columns = ['city', 'state', 'population']

counties_population_data['city'] = counties_population_data['city'].str.replace(
  r' (city|town|village|County|Parish|Planning Region|Census Area|City and Borough|Borough|Municipality)$',
  '', regex=True)
counties_population_data['city'] = counties_population_data['city'].str.replace(r'\bSt\. \b', 'Saint ', regex=True)
counties_population_data['city'] = counties_population_data['city'].str.strip()

In [29]:
# Preparing and Cleaning SUMLEV 061(Towns) Population Data
towns_population_data = population_data[population_data["SUMLEV"] == 61]
towns_population_data = towns_population_data[['NAME', 'STNAME', 'POPESTIMATE2022']]
towns_population_data.columns = ['city', 'state', 'population']
towns_population_data['city'] = towns_population_data['city'].str.replace(
  r' (City|city|township|town|village|-Troy Hills)', '', regex=True)
towns_population_data['city'] = towns_population_data['city'].str.replace(r'\bSt\. \b', 'Saint ', regex=True)

In [30]:
# Cleaning and preparing Louisville(SUMLEV 157) and Nashville(SUMLEV 172) data
targeted_population_data = population_data[population_data["SUMLEV"].isin([157, 172])]
targeted_population_data = targeted_population_data[['NAME', 'STNAME', 'POPESTIMATE2022']]
targeted_population_data.columns = ['city', 'state', 'population']

targeted_population_data['city'] = targeted_population_data['city'].str.replace(
  r'(-Davidson metropolitan government \(balance\)|/Jefferson County metro government \(balance\))', '', regex=True)

In [31]:
#Preparing and cleaning S&P500 Data
split_locations = snp500_data['Headquarters Location'].str.split(',', expand=True)
snp500_data['city'] = split_locations[0]
snp500_data['state'] = split_locations[1] if split_locations.shape[1] > 1 else None

snp500_data['city'] = snp500_data['city'].str.strip()
snp500_data['state'] = snp500_data['state'].str.strip() if snp500_data['state'] is not None else None
snp500_data['city'] = snp500_data['city'].str.replace(r' (City|County|Village|Ranch)$', '', regex=True)
snp500_data['city'] = snp500_data['city'].str.replace(r'\bSt\. \b', 'Saint ', regex=True)
if 'state' in snp500_data.columns:
  snp500_data['state'] = snp500_data['state'].str.replace(r'\bD.C\.', 'District of Columbia', regex=True)

In [32]:
#Intial Merge with cities data
snp500_with_population = pd.merge(snp500_data, cities_population_data, on=['city', 'state'], how='left')


In [33]:
# Function to check for NANs and fill them with population_data
def fill_population(row):
	if pd.isna(row['population']):
		# Try to get the population from the county data
		county_population = counties_population_data[
			(counties_population_data['city'] == row['city']) &
			(counties_population_data['state'] == row['state'])
			]['population'].values
		if len(county_population) > 0:
			return county_population[0]
		# If County data not found try Towns data
		town_population = towns_population_data[
			(towns_population_data['city'] == row['city']) &
			(towns_population_data['state'] == row['state'])
		]['population'].values
		if len(town_population) > 0:
			return town_population[0]
		targeted_population = targeted_population_data[
			(targeted_population_data['city'] == row['city']) &
			(targeted_population_data['state'] == row['state'])
			]['population'].values
		return targeted_population[0] if len(targeted_population) > 0 else None
	return row['population']

snp500_with_population['population'] = snp500_with_population.apply(fill_population, axis=1)

path_to_save_csv = '../Group3Project1/Resources/snp500_with_population.csv'
snp500_with_population.to_csv(path_to_save_csv, index=False)

In [34]:
#print(snp500_with_population.info())
print(snp500_with_population['population'].isna().sum())

51


In [35]:
#Creating CSV with NAN population values rows removed
snp500_no_nan_population = snp500_with_population.dropna(subset=['population'])

snp500_no_nan_population_path = '../Group3Project1/Resources/snp500_no_nan_population.csv'
snp500_no_nan_population.to_csv(snp500_no_nan_population_path, index=False)

In [36]:
# Adding PCI(Per Capita Income) to snp500_no_nan_population.csv
# Reading in snp500 data.
snp500_no_nan_data = pd.read_csv(snp500_no_nan_population_path)

# Defining Path and reading in PCI data
PCI_path = '../Group3Project1/Resources/CIANC1_ALL_AREAS_2021.csv'
pci_2021_data = pd.read_csv(PCI_path)

In [37]:
# Preparing data
# Split GeoName Into city and state column
pci_2021_data[['city_pci','state']] = pci_2021_data['GeoName'].str.rsplit(',', n=1, expand=True)

# Strip any whitespace and * from the 'state' column
pci_2021_data['state'] = pci_2021_data['state'].str.replace('*', '').str.strip()

# Creating first_word column
pci_2021_data['first_word'] = pci_2021_data['city_pci'].str.split().str[0]

# Creating Dictionary to change 'state' column state initials to full state name
state_name = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC':'District of Columbia'
}

pci_2021_data['state'] = pci_2021_data['state'].map(state_name)

# Create first_word column for snp_500_no_nan_data
snp500_no_nan_data['first_word'] = snp500_no_nan_data['city'].str.split().str[0]

In [38]:
# Merging Data on 'state' and 'first_word'
snp500_with_pci = pd.merge(snp500_no_nan_data, pci_2021_data, on=['state','first_word'], how='left')
display(snp500_with_pci.head(10))
# Keep only revelent columns from pci_2021_data
pci_columns = ['Description', 'Unit', '2021', 'city_pci']
snp500_with_pci = snp500_with_pci[snp500_no_nan_data.columns.tolist() + pci_columns]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,city,state,...,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2021,city_pci
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,Saint Paul,Minnesota,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,Milwaukee,Wisconsin,...,"""55079""","Milwaukee, WI",3,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,51904010,Milwaukee
2,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,Milwaukee,Wisconsin,...,"""55079""","Milwaukee, WI",3,CAINC1,2.0,...,Population (persons) 1/,Number of persons,928059,Milwaukee
3,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,Milwaukee,Wisconsin,...,"""55079""","Milwaukee, WI",3,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,55927,Milwaukee
4,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,North Chicago,Illinois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),North Chicago,Illinois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ADM,ADM,Consumer Staples,Agricultural Products & Services,"Chicago, Illinois",1957-03-04,7084,1902,Chicago,Illinois,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982,San Jose,California,...,"""06069""","San Benito, CA",8,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,4591936,San Benito
8,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982,San Jose,California,...,"""06069""","San Benito, CA",8,CAINC1,2.0,...,Population (persons) 1/,Number of persons,66677,San Benito
9,ADBE,Adobe Inc.,Information Technology,Application Software,"San Jose, California",1997-05-05,796343,1982,San Jose,California,...,"""06069""","San Benito, CA",8,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,68868,San Benito


In [39]:
# Check for Non matches,
# Identifying non-matching companies (without PCI data)
non_match_companies = snp500_with_pci[snp500_with_pci['2021'].isnull()]

# Creating a list of unique cities from the PCI data
cities_in_pci_data = pci_2021_data['city_pci'].unique()

# Finding the cities from the non-matching companies that are not in the PCI data
cities_not_in_pci = non_match_companies[~non_match_companies['city'].isin(cities_in_pci_data)]['city'].unique()

# Count the number of companies from the S&P 500 dataset that are located in these cities not in the PCI data
count_companies_in_cities_not_in_pci = non_match_companies[non_match_companies['city'].isin(cities_not_in_pci)].shape[0]

print(count_companies_in_cities_not_in_pci)

241


In [40]:
# Filtering for wanted information and Merging
# Filter the PCI data for the relevant rows
filtered_pci_data = pci_2021_data[(pci_2021_data['Description'] == 'Per capita personal income (dollars) 2/') &
                                  (pci_2021_data['Unit'] == 'Dollars')]

# Merge the filtered PCI data with the S&P 500 data
snp500_with_pci = pd.merge(snp500_no_nan_data, filtered_pci_data, left_on=['city', 'state'], right_on=['city_pci', 'state'], how='left')

# Rename the '2021' column to '2021_pci'
snp500_with_pci.rename(columns={'2021': '2021_pci'}, inplace=True)

# Remove unwanted columns
columns_to_drop = ['first_word_y', 'city_pci', 'IndustryClassification', 'LineCode', 'TableName', 'Region', 'GeoName', 'GeoFIPS', 'first_word_x']
snp500_with_pci.drop(columns=columns_to_drop, inplace=True)

snp500_with_pci.to_csv('../Group3Project1/Resources/snp500_with_pci.csv', index=False)


In [41]:
# Adding Marketcap to Snp500_with_pci CSV
# Defining Path
snp500_with_pci_path = Path('../Group3Project1/Resources/snp500_with_pci.csv')
snp500_marketcap_path = Path('../Group3Project1/Resources/snp500_marketcap.csv')
# Readin in Csv
snp500_with_pci = pd.read_csv(snp500_with_pci_path)
snp500_marketcap = pd.read_csv(snp500_marketcap_path)

In [42]:
# Rename Ticker column to Symbol
snp500_marketcap.rename(columns={'Ticker':'Symbol'}, inplace=True)

In [49]:
# Merged Datasets
snp500_marketcap_merged = pd.merge(snp500_with_pci, snp500_marketcap, on='Symbol')

# Drop unwanted columns
snp500_marketcap_merged = snp500_marketcap_merged.drop(columns=['Description_y', 'Sector'])

# Save Dataframe to CSV
snp500_marketcap_merged.to_csv('../Group3Project1/Resources/snp500_pci_marketcap.csv', index=False)